In [1]:
import numpy as np
import torch.nn as nn
import torch
import torchvision

In [ ]:
import time, sys, string, os

from utils.func_utils import accept, jacobian, autocovariance, get_log_likelihood, get_data,\
    var_from_scope, binarize, normal_kl, binarize_and_shuffle
from utils.distributions import Gaussian, GMM, GaussianFunnel, gen_ring
from utils.layers import Linear, Parallel, Sequential, Zip, ScaleTanh
from utils.dynamics import Dynamics
from utils.sampler import propose, tf_accept, chain_operator
from utils.losses import get_loss, loss_mixed

FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('hparams', '', 'Comma sep list of name=value')
tf.app.flags.DEFINE_string('exp_id', '', 'exp_id')

DEFAULT_HPARAMS = tf.contrib.training.HParams(
    learning_rate=0.001,
    epoch=100,
    leapfrogs=5,
    MH=5,
    optimizer='adam',
    batch_size=512,
    latent_dim=50,
    update_sampler_every=1,
    eval_samples_every=1,
    random_lf_composition=0,
    stop_gradient=False,
    hmc=False,
    eps=0.1,
    energy_scale=0.,
)

# hardcode the loss
LOSS = 'mixed'

OPTIMIZERS = {
    'adam': tf.train.AdamOptimizer,
    'rmsprop': tf.train.RMSPropOptimizer,
    'nesterov': tf.train.MomentumOptimizer,
    'sgd': tf.train.GradientDescentOptimizer,
}